In [12]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ModuleNotFoundError: No module named 'pandas'

In [13]:
os.listdir('data/')

NameError: name 'os' is not defined

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head(2)

In [ ]:
# here we can find train data have many empty value
pd.DataFrame(train_df.isnull().sum()/train_df.shape[0]).transpose()

In [ ]:
# here we can find test data have many empty value
pd.DataFrame(test_df.isnull().sum()/test_df.shape[0]).transpose()

In [ ]:
# extract sale price first
original_y = train_df['SalePrice']
train_df.drop('SalePrice', axis=1, inplace=True)

In [ ]:
# we can concat the train and test data to handle empty value together
data= pd.concat([train_df,test_df], keys=['train', 'test'])
data=data.drop(["Id"],axis=1)

In [ ]:
# check the null value
empty = pd.DataFrame(data.isnull().sum()/data.shape[0])
empty =  empty[empty[0]!=0] # ignore the columns without empty value
empty.sort_values(by=0, ascending=False) # sort the value

In [ ]:
# drop the columns have more than 20% empty values
data.drop(empty[empty[0] > 0.2].index.to_list(), axis=1, inplace=True)

In [ ]:
# fill na values
for col in data.select_dtypes(include=['number']).columns:
    if data[col].isnull().sum() == 0:
        continue
    else:
        data[col].fillna(data[col].mean(),inplace=True)
        
for col in data.select_dtypes(include=['object']).columns:
    if data[col].isnull().sum() == 0:
        continue
    else:
        data[col].fillna(data[col].mode()[0],inplace=True)

In [ ]:
# check 
pd.DataFrame(data.isnull().sum()/data.shape[0]).transpose()

In [ ]:
# feature selection 
data.shape

In [ ]:
# convert objects type to numeric type
for col in data.select_dtypes(include=['object']).columns:
    data[col] = pd.factorize(data[col])[0]

In [11]:
# we have 74 features, so we need find the correlation for them
correlation_score = []
col = data.columns.to_list()
fig, axs = plt.subplots(4, 4, figsize=(8, 8),sharey=True)
axs = axs.flatten()
train = data.loc['train']
for i in range(16):
    cor = np.corrcoef(train.iloc[:, i].values, original_y.values)[0][1]
    correlation_score.append(cor)
    axs[i].scatter(train.iloc[:, i].values, original_y.values)
    axs[i].legend([f'cor: {cor:.2f}'])
    axs[i].set_xlabel(col[i])
plt.tight_layout()

NameError: name 'data' is not defined

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(8, 8),sharey=True)
axs = axs.flatten()
for i in range(16):
    cor = np.corrcoef(train.iloc[:, i+16].values, original_y.values)[0][1]
    correlation_score.append(cor)
    axs[i].scatter(train.iloc[:, i+16].values, original_y.values)
    axs[i].legend([f'cor: {cor:.2f}'])
    axs[i].set_xlabel(col[i])
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(8, 8),sharey=True)
axs = axs.flatten()
for i in range(16):
    cor = np.corrcoef(train.iloc[:, i+16*2].values, original_y.values)[0][1]
    correlation_score.append(cor)
    axs[i].scatter(train.iloc[:, i+16*2].values, original_y.values)
    axs[i].legend([f'cor: {cor:.2f}'])
    axs[i].set_xlabel(col[i])
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(8, 8),sharey=True)
axs = axs.flatten()
for i in range(16):
    cor = np.corrcoef(train.iloc[:, i+16*3].values, original_y.values)[0][1]
    correlation_score.append(cor)
    axs[i].scatter(train.iloc[:, i+16*3].values, original_y.values)
    axs[i].legend([f'cor: {cor:.2f}'])
    axs[i].set_xlabel(col[i])
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(8, 8),sharey=True)
axs = axs.flatten()
for i in range(16*4, data.shape[1]):
    cor = np.corrcoef(train.iloc[:, i].values, original_y.values)[0][1]
    correlation_score.append(cor)
    axs[i-16*4].scatter(train.iloc[:, i].values, original_y.values)
    axs[i-16*4].legend([f'cor: {cor:.2f}'])
    axs[i-16*4].set_xlabel(col[i])
for i in range(10, 16):
    axs[i].axis('off')
plt.tight_layout()

In [ ]:
# from the picture above, we can see more feature have a very low relationship will label saleprice
# so we need discharge those feature, only focus on the feature have strong relation ship will label
# here i think the abs correlationship value < 0.2  mean have no relationship, value between 0.2 and  0.6 mean have weak relationship
# and value greater than 0.6 have strong relationship, so we need delete the feature have correlationship small than 0.2
step2_data = data.loc[:, np.array(correlation_score) > 0.2]

In [ ]:
# check the shape, here we find we have delete 24 feature 
step2_data.shape

In [ ]:
# check the feature correlationship with each other, we can find many feature pair have strong relationship
# each many cause series Collinearity
plt.figure(figsize=(12,9))
sns.heatmap(step2_data.corr())

In [ ]:
# convert outlines points in range
fig, axs = plt.subplots(6, 4, figsize=(12, 15))
axs = axs.flatten()
col = step2_data.columns.to_list()
string_col = data.select_dtypes(include=['object']).columns
for col_index in range(step2_data.shape[1]):
    if col[col_index] in string_col:
        axs[col_index].boxplot(step2_data.iloc[:, col_index], c='red',patch_artist = True)
    else:
        axs[col_index].boxplot(step2_data.iloc[:, col_index])
    axs[col_index].set_xlabel(col[col_index])
plt.tight_layout()

In [ ]:
def top_bottom(x):
    Q3 = np.percentile(x, 75)
    Q1 = np.percentile(x, 25)
    LQR = Q3-Q1
    return Q1-1.5*LQR, Q3+1.5*LQR

for col in step2_data.columns.to_list():
    if col not in string_col:
        bottom, top = top_bottom(step2_data[col])
        step2_data[col][step2_data[col] < bottom] = bottom
        step2_data[col][step2_data[col] > top] = top

In [ ]:
# check the feature distribution after convert outline process
fig, axs = plt.subplots(6, 4, figsize=(12, 15))
axs = axs.flatten()
col = step2_data.columns.to_list()
string_col = data.select_dtypes(include=['object']).columns
for col_index in range(step2_data.shape[1]):
    if col[col_index] in string_col:
        axs[col_index].boxplot(step2_data.iloc[:, col_index], c='red',patch_artist = True)
    else:
        axs[col_index].boxplot(step2_data.iloc[:, col_index])
    axs[col_index].set_xlabel(col[col_index])
plt.tight_layout()

In [ ]:
# we can see the distribution of the features
# we need do scale transform
fig, axs = plt.subplots(6, 4, figsize=(12, 15))
axs = axs.flatten()
col = step2_data.columns.to_list()
for col_index in range(step2_data.shape[1]):
    axs[col_index].hist(step2_data.iloc[:, col_index])
    axs[col_index].set_xlabel(col[col_index])
plt.tight_layout()

In [ ]:
# make all feature value range in 0-1
std = MinMaxScaler()
train = step2_data.loc['train']
test = step2_data.loc['test']
std.fit(train)
train = std.transform(train)
test = std.transform(test)

In [ ]:
# model selection, here i try to use linear model include lr, lasso ridge
# and neural new work MLP regression and ensemble random forest regressor
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
#train = step2_data.loc['train']
train_x, val_x, train_y, val_y = train_test_split(train, original_y, random_state=0, test_size=0.2)

In [ ]:
# first mdoel  lr
r2_list = []
rmse = []
lr = LinearRegression()
lr.fit(train_x, train_y)
r2_list.append(lr.score(val_x,val_y))
rmse.append(mean_squared_error(lr.predict(val_x), val_y)**0.5)

In [ ]:
ls =  LassoCV()
ls .fit(train_x, train_y)
r2_list.append(ls .score(val_x,val_y))
rmse.append(mean_squared_error(ls.predict(val_x), val_y)**0.5)

In [ ]:
ri =  RidgeCV()
ri .fit(train_x, train_y)
r2_list.append(ri .score(val_x,val_y))
rmse.append(mean_squared_error(ri.predict(val_x), val_y)**0.5)

In [ ]:
rft = RandomForestRegressor()
rft .fit(train_x, train_y)
r2_list.append(rft .score(val_x,val_y))
rmse.append(mean_squared_error(rft.predict(val_x), val_y)**0.5)

In [ ]:
# the best r2 is random forest regressor
# so we need do parameters tunning
params = {
    'n_estimators':[100,300, 500],
    'max_depth': [None, 1, 2, 5, 8],
    'bootstrap':[True, False]
}
rtf = GridSearchCV(RandomForestRegressor(), param_grid=params, cv=5, n_jobs=5)
rtf.fit(train_x, train_y)
rft = RandomForestRegressor(**rtf.best_params_)
rft .fit(train_x, train_y)
r2_list.append(rft .score(val_x,val_y))
rmse.append(mean_squared_error(rft.predict(val_x), val_y)**0.5)

In [ ]:
r2_list

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,6))
axs = axs.flatten()
content = '1.LinearRegression' + '\n' + '2.Lasso' + '\n' + \
    '3.Ridge' + '\n' + '4.RandomForestRegressor' + '\n' + '5.RandomForestRegressor tunning'
axs[0].plot(range(5), r2_list, marker='s')
axs[0].legend([content])
axs[1].plot(range(5), rmse, marker='o', linestyle='-.', c='r')
axs[1].legend([content])

In [ ]:
# use model 
test_pred = rtf.predict(test)

In [ ]:
# prediction result have very similiar distribution with original y, so the prediction result is Credible
temp_y = np.vstack([test_pred.reshape(-1, 1), original_y.values.reshape(-1, 1)])
hue = ['test']*len(test_pred) + ['train'] *len(original_y)
temp_df = pd.DataFrame(columns=['y','hue'], index=range(len(hue)))
temp_df['y'] = temp_y
temp_df['hue'] = hue
plt.figure(figsize=(10, 7))
ax = plt.gca()
sns.histplot(data=temp_df, x='y', hue='hue',kde=True, ax=ax,element="step")